In [2]:
#as of now i assume that taking much more bigger dataset would be better so for testing purpose i have taken new data set from kaggle competition as well 
#i have taken both the data set for operation , the above mentioned one and the one which my team had provided

In [20]:
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np

In [21]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk as nltk
#import nltk.corenlp as nlp
import seaborn as sbn
import nltk.grammar as grm
import nltk.corpus as corpus

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.linear_model import PassiveAggressiveClassifier, LinearRegression
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVR,SVC
import seaborn as sbn


import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier

In [22]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D
from string import punctuation

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as pl

In [23]:
df_1=pd.read_pickle("/home/rd/Desktop/prj/cleaned_df.pkl")


In [24]:
print(df_1.shape)
df_1.head()

(470, 8)


,link,published,title,text,author,label,clean_title,clean_text
0,https://www.nytimes.com/2019/11/20/business/me...,2019-11-21T04:22:14,Media Workers Call Out Pay Gaps in Crowdsource...,With income inequality a focus of the current ...,"[Marc Tracy, Tiffany Hsu]",0,media workers call pay gaps crowdsourced sprea...,income inequality focus current presidential c...
1,https://www.nytimes.com/2019/11/20/business/gm...,2019-11-21T00:07:24,G.M. Sues Rival Over Bribery Scheme as Union S...,Fiat Chrysler “was able to obtain unique advan...,[Neal E. Boudette],0,gm sues rival bribery scheme union scandal exp...,"fiat chrysler ""was able obtain unique advantag..."
2,https://www.nytimes.com/2019/11/20/business/ho...,2019-11-21T01:58:42,U.S. Bill Supporting Hong Kong Rights Heads to...,A bill compelling the United States to support...,"[David Yaffe-Bellany, Alan Rappeport]",0,us bill supporting hong kong rights heads trum...,bill compelling united states support pro demo...
3,https://www.nytimes.com/2019/11/20/business/tr...,2019-11-21T02:08:37,Trump Floats Tariff Exclusions for Apple at Te...,"Jonathan Gold, a spokesman for Americans for F...",[Ana Swanson],0,trump floats tariff exclusions apple texas man...,jonathan gold spokesman americans free trade l...
4,https://www.nytimes.com/2019/11/21/business/de...,2019-11-21T08:00:10,Henry Paulson Sounds Alarm: U.S.-China Relatio...,The United States and China will eventually se...,[Andrew Ross Sorkin],0,henry paulson sounds alarm us china relations ...,united states china eventually settle differen...


In [25]:
df_1.isnull().sum()
#yippie no missing values ! if its zero

link           0
published      0
title          0
text           0
author         0
label          0
clean_title    0
clean_text     0
dtype: int64

In [26]:
length = []
[length.append(len(str(text))) for text in df_1['text']]
df_1['length'] = length
df_1.head()

min(df_1['length']), max(df_1['length']), round(sum(df_1['length'])/len(df_1['length']))

(51, 33547, 3613)

In [27]:
#does it mean that we can keep 3600 features in nn
#51 kahi ye outliers to nhi 
#just in case to verify this 


In [28]:
df_1.head()

,link,published,title,text,author,label,clean_title,clean_text,length
0,https://www.nytimes.com/2019/11/20/business/me...,2019-11-21T04:22:14,Media Workers Call Out Pay Gaps in Crowdsource...,With income inequality a focus of the current ...,"[Marc Tracy, Tiffany Hsu]",0,media workers call pay gaps crowdsourced sprea...,income inequality focus current presidential c...,1213
1,https://www.nytimes.com/2019/11/20/business/gm...,2019-11-21T00:07:24,G.M. Sues Rival Over Bribery Scheme as Union S...,Fiat Chrysler “was able to obtain unique advan...,[Neal E. Boudette],0,gm sues rival bribery scheme union scandal exp...,"fiat chrysler ""was able obtain unique advantag...",3104
2,https://www.nytimes.com/2019/11/20/business/ho...,2019-11-21T01:58:42,U.S. Bill Supporting Hong Kong Rights Heads to...,A bill compelling the United States to support...,"[David Yaffe-Bellany, Alan Rappeport]",0,us bill supporting hong kong rights heads trum...,bill compelling united states support pro demo...,1303
3,https://www.nytimes.com/2019/11/20/business/tr...,2019-11-21T02:08:37,Trump Floats Tariff Exclusions for Apple at Te...,"Jonathan Gold, a spokesman for Americans for F...",[Ana Swanson],0,trump floats tariff exclusions apple texas man...,jonathan gold spokesman americans free trade l...,1887
4,https://www.nytimes.com/2019/11/21/business/de...,2019-11-21T08:00:10,Henry Paulson Sounds Alarm: U.S.-China Relatio...,The United States and China will eventually se...,[Andrew Ross Sorkin],0,henry paulson sounds alarm us china relations ...,united states china eventually settle differen...,1035


In [29]:
len(df_1[df_1['length'] < 50])

0

In [30]:
#saved ! if its 0

In [31]:
df_1['text'][df_1['length'] < 50]

Series([], Name: text, dtype: object)

In [32]:
max_features=3600

In [33]:
#as the data set provided by my team is already preprocessed so i'm not proceeding with this
x = df_1['clean_text']
from nltk.tokenize import RegexpTokenizer
import time


tokenizer = RegexpTokenizer(r'\w+')
x = x.astype('str') 
x.dtypes

x = x.apply(tokenizer.tokenize)
# delete Stop Words

#print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
x

0      [income, inequality, focus, current, president...
1      [fiat, chrysler, was, able, obtain, unique, ad...
2      [bill, compelling, united, states, support, pr...
3      [jonathan, gold, spokesman, americans, free, t...
4      [united, states, china, eventually, settle, di...
                             ...                        
465    [looking, student, loan, might, definitely, he...
466    [world, growing, digital, currencies, digital,...
467    [love, traveling, would, surely, heard, new, b...
468    [indigo, card, login, indigo, credit, cards, h...
469    [mybpcreditcard, login, bp, credit, card, issu...
Name: clean_text, Length: 470, dtype: object

In [34]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = df_1['text'])
x = tokenizer.texts_to_sequences(texts = df_1['text'])

In [35]:
x = pad_sequences(sequences = x, maxlen = max_features, padding = 'pre',dtype=float)

In [36]:
#x=df_1['clean_text']
y=df_1['label']

In [37]:
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.70,random_state=111)


In [38]:
lstm_model = Sequential(name = 'lstm_nn_mo')

#layer 1 lstm
lstm_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
#l2
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
#l3
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
#l4
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
#l5
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
#sigmoid 
lstm_model.add(layer = Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
lstm_model_fit = lstm_model.fit(train_x, train_y, epochs = 1)
#epochs to be 20+ atleast , try when network good on collab

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
#now i split my dataset into two section
#i have already made segregations for the test and training set from df_1
import multiprocessing as mp
cores = multiprocessing.cpu_count() 
# Count the number of cores in a computer, important for a parameter of the model
train_model = train_x(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
train_model.build_vocab(x, progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
train_model.train(x, total_examples=train_x.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))